In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
text=pd.read_csv('/content/drive/MyDrive/DUC 2002 A/Input Files/DUC 2002 Abstractive.csv')

In [4]:
pretext=pd.read_csv('/content/drive/MyDrive/DUC 2002 A/Input Files/NSW.csv')

In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import networkx as nx

def preprocess(text):
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]
    #print(words)
    words = [word for sentence in words for word in sentence]
    #print("\n")
    #print(words)
    return words

import gensim.downloader as api

def build_graph(words): 
    model = api.load('word2vec-google-news-300')

    graph = nx.Graph()
    for i, word in enumerate(words):
        for j in range(i + 1, len(words)):
            if word != words[j]:
                if word in model.key_to_index and words[j] in model.key_to_index:
                    similarity = np.dot(model.get_vector(word), model.get_vector(words[j])) / (np.linalg.norm(model.get_vector(word)) * np.linalg.norm(model.get_vector(words[j])))
                    if similarity > 0:
                        graph.add_edge(word, words[j], weight=similarity)

    return graph

def summarize(text,pretext, num_sentences):
    words = preprocess(pretext)
    graph = build_graph(words)

  
    scores = nx.pagerank(graph)

    
    ranked_sentences = sorted(((scores[word], sentence) for i, sentence in enumerate(sent_tokenize(text)) for word in word_tokenize(sentence.lower()) if word in scores), reverse=True)
    summary = ' '.join([sentence for score, sentence in ranked_sentences[:num_sentences]])

    return summary


In [6]:
#text="This is a long piece of text that contains many sentences. The goal of this code is to summarize the text using a graph-based approach. The TextRank algorithm is used to rank the sentences based on their importance. The top sentences are then selected to create a summary. The result is a concise representation of the original text."

In [7]:
print("\n\nSummary:\n")
print(summarize(text.iloc[58][2],pretext.iloc[58][2],num_sentences=6))



Summary:

officials decided two weeks ago to remove Checkpoint Charlie.The border crossing was the scene of stirring escapes andheartbreaking captures as East Germans tried flee to the West,breaking through East German control stations just 20 yards awayfrom the Allied checkpoint.East Germans once manned their side of the checkpoint withemotionless effiency, closely scrutinizing travel documents andcreating a palpable tension. officials decided two weeks ago to remove Checkpoint Charlie.The border crossing was the scene of stirring escapes andheartbreaking captures as East Germans tried flee to the West,breaking through East German control stations just 20 yards awayfrom the Allied checkpoint.East Germans once manned their side of the checkpoint withemotionless effiency, closely scrutinizing travel documents andcreating a palpable tension. officials decided two weeks ago to remove Checkpoint Charlie.The border crossing was the scene of stirring escapes andheartbreaking captures as Ea